In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

def final_automated_scrape():
    url = "https://www.oddset.de/de/sports/radsport-10/wetten/welt-6/tour-de-france-160"
    
    chrome_options = Options()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    print(f"Attempting to open: {url}")
    driver.get(url)

    # --- Handle Cookie Consent Banner ---
    try:
        print("Looking for cookie consent button...")
        cookie_button_id = "onetrust-accept-btn-handler"
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, cookie_button_id))).click()
        print("Cookie banner accepted.")
        time.sleep(3)
    except Exception:
        print("Did not find or need to click the cookie button.")

    # --- FINAL ATTEMPT: Find button by its text using XPath ---
    try:
        # XPath is a powerful way to find elements by their visible text.
        # This looks for any clickable element (button or div) that contains the text "Mehr anzeigen".
        show_more_xpath = "//*[contains(text(), 'Mehr anzeigen')]"
        print(f"Finding all 'Mehr anzeigen' buttons with XPath: {show_more_xpath}")

        # Find all buttons that match the selector
        show_more_buttons = driver.find_elements(By.XPATH, show_more_xpath)
        print(f"Found {len(show_more_buttons)} 'Mehr anzeigen' buttons to click.")

        if not show_more_buttons:
             print("Could not find any 'Mehr anzeigen' buttons automatically.")

        for i, button in enumerate(show_more_buttons):
            try:
                driver.execute_script("arguments[0].click();", button)
                print(f"Clicked button #{i+1}. Waiting...")
                time.sleep(2)
            except Exception as e:
                print(f"Could not click button #{i+1}. Error: {e}")
        
    except Exception as e:
        print(f"An error occurred while trying to click 'Mehr anzeigen' buttons: {e}")

    # --- SCRAPING LOGIC ---
    # ... (The rest of the script is the same, it will scrape whatever is visible)
    rider_odds = []
    try:
        rider_option_selector = "ms-option.option"
        print(f"\nNow scraping all rider options with selector: '{rider_option_selector}'")
        rider_options = driver.find_elements(By.CSS_SELECTOR, rider_option_selector)
        print(f"Found {len(rider_options)} individual rider options on the page.")

        for option in rider_options:
            try:
                rider_name = option.find_element(By.CSS_SELECTOR, "div.name").text.strip()
                odds_value = option.find_element(By.CSS_SELECTOR, "div.value").text.strip()
                if rider_name and odds_value:
                    rider_odds.append({"Rider": rider_name, "Odds": odds_value})
            except Exception:
                continue
    
    finally:
        driver.quit()

    if rider_odds:
        print("\n--- ✅ Scraped Data ---")
        df = pd.DataFrame(rider_odds).drop_duplicates(subset=['Rider'])
        df_final = df[df['Rider'] != 'Alle anderen']
        print(df_final)
        print(f"\nTotal unique riders scraped: {len(df_final)}")
        return df_final
    else:
        print("\n--- ❌ No Data Was Scraped ---")
        return pd.DataFrame()

# --- Run the final automated attempt ---
final_automated_scrape()

Attempting to open: https://www.oddset.de/de/sports/radsport-10/wetten/welt-6/tour-de-france-160
Looking for cookie consent button...
Cookie banner accepted.
Finding all 'Mehr anzeigen' buttons with XPath: //*[contains(text(), 'Mehr anzeigen')]
Found 2 'Mehr anzeigen' buttons to click.
Clicked button #1. Waiting...
Clicked button #2. Waiting...

Now scraping all rider options with selector: 'ms-option.option'
Found 55 individual rider options on the page.

--- ✅ Scraped Data ---
                               Rider     Odds
0                Tadej Pogacar (SLO)     1.35
1             Jonas Vingegaard (DEN)     3.25
2              Remco Evenepoel (BEL)    15.00
3                 Joao Almeida (POR)    21.00
4                Primoz Roglic (SLO)    34.00
5             Florian Lipowitz (GER)    34.00
6             Matteo Jorgenson (USA)    67.00
7                   Felix Gall (AUT)    67.00
8            Mattias Skjelmose (DEN)   101.00
9                  Simon Yates (GBR)   101.00
10        

,Rider,Odds
0,Tadej Pogacar (SLO),1.35
1,Jonas Vingegaard (DEN),3.25
2,Remco Evenepoel (BEL),15.00
3,Joao Almeida (POR),21.00
4,Primoz Roglic (SLO),34.00
5,Florian Lipowitz (GER),34.00
6,Matteo Jorgenson (USA),67.00
7,Felix Gall (AUT),67.00
8,Mattias Skjelmose (DEN),101.00
9,Simon Yates (GBR),101.00
